In [45]:
import numpy as np
import pandas as pd

data = pd.read_csv("data/fifa_2019_player_data.csv", encoding="Windows-1252")

In [46]:
# Modeli bozacak sütunları kaldırdım.
data.drop(columns=["Index", "Jersey Number", "Contract Valid Until", "Preferred Foot"], inplace=True)
data.info()
data.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18207 entries, 0 to 18206
Data columns (total 85 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        18207 non-null  int64  
 1   Name                      18207 non-null  object 
 2   Age                       18206 non-null  float64
 3   Photo                     18207 non-null  object 
 4   Nationality               18207 non-null  object 
 5   Flag                      18207 non-null  object 
 6   Overall                   18206 non-null  float64
 7   Potential                 18207 non-null  int64  
 8   Club                      17966 non-null  object 
 9   Club Logo                 18207 non-null  object 
 10  Value                     18207 non-null  object 
 11  Wage                      18207 non-null  object 
 12  Special                   18207 non-null  int64  
 13  International Reputation  18159 non-null  float64
 14  Weak F

,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,158023,L. Messi,31.0,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94.0,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,20801,Cristiano Ronaldo,33.0,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94.0,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,...,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,190871,Neymar Jr,26.0,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92.0,93,Paris Saint-Germain,https://cdn.sofifa.org/teams/2/light/73.png,...,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,193080,De Gea,27.0,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91.0,93,Manchester United,https://cdn.sofifa.org/teams/2/light/11.png,...,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,192985,K. De Bruyne,27.0,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91.0,92,Manchester City,https://cdn.sofifa.org/teams/2/light/10.png,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


In [47]:
new_data = pd.DataFrame(data[["Name", "Overall"]])
for column in data.columns:
    if (column in new_data.columns):
        continue
    try:
        new_data[column] = data[column].astype(float)
    except:
        continue

In [48]:
# Kayıp verileri daha sonrasında kullanmak için kaydediyorum.
missing_data = new_data[new_data["Overall"].isna()]

missing_data.drop("Overall", axis=1, inplace=True)

C:\Users\mucakmak\AppData\Local\Temp\ipykernel_7052\2025339627.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_data.drop("Overall", axis=1, inplace=True)


In [49]:
# Nan değerleri doldurma işlemi
from sklearn.impute import SimpleImputer

numeric_data = new_data.select_dtypes(include=[np.number])
imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
imputer = imputer.fit(numeric_data)
new_data[numeric_data.columns] = imputer.transform(numeric_data)
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18207 entries, 0 to 18206
Data columns (total 43 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Name                      18207 non-null  object 
 1   Overall                   18207 non-null  float64
 2   ID                        18207 non-null  float64
 3   Age                       18207 non-null  float64
 4   Potential                 18207 non-null  float64
 5   Special                   18207 non-null  float64
 6   International Reputation  18207 non-null  float64
 7   Weak Foot                 18207 non-null  float64
 8   Skill Moves               18207 non-null  float64
 9   Crossing                  18207 non-null  float64
 10  Finishing                 18207 non-null  float64
 11  HeadingAccuracy           18207 non-null  float64
 12  ShortPassing              18207 non-null  float64
 13  Volleys                   18207 non-null  float64
 14  Dribbl

In [50]:
# One-Hot Encoding | Burası score değerini 0.001 düşürüyor bu yüzden yorum satirinda
# from sklearn import preprocessing

# # Preferred Foot one-hot encoding işlemi
# new_data.dropna(subset=["Preferred Foot"], inplace=True)
# ohe = preprocessing.OneHotEncoder(sparse_output=False)
# preferred_foot = ohe.fit_transform(new_data["Preferred Foot"].values.reshape(-1, 1))
# preferred_foot_df = pd.DataFrame(preferred_foot, columns=ohe.get_feature_names_out(['Preferred Foot']))

# # Concat OHE DataFrame
# new_data = pd.concat([new_data.reset_index(drop=True), preferred_foot_df.reset_index(drop=True)], axis=1)

# # Eklenen datayı kaldırma
# new_data.drop("Preferred Foot", axis=1, inplace=True)

In [51]:
# Modeli train test olarak ayırma işlemi.
from sklearn.model_selection import train_test_split

y_value = pd.DataFrame(new_data["Overall"].values)
x_values = new_data.drop(columns=["Overall", "Name"]).copy()

x_train, x_test, y_train, y_test = train_test_split(x_values, y_value, test_size=0.33, random_state=23)

In [52]:
# Modeli Multiple Linear Regresyon modeline göre eğitme 
from sklearn.linear_model import LinearRegression

regression = LinearRegression()
regression.fit(x_train, y_train)

y_pred_linear = regression.predict(x_test)

In [ ]:
# Model doğruluk analizi
y_pred_df = pd.DataFrame(y_pred_linear.flatten(), columns=['Predicted'])
y_test_df = pd.DataFrame(y_test.values, columns=['Actual'])
pred_and_test_data = pd.concat([y_pred_df, y_test_df], axis=1)

accuracy = regression.score(x_test, y_test)
print("Doğruluk oranı: ", accuracy)
print(pred_and_test_data)

Doğruluk oranı:  0.92796242597343
      Predicted  Actual
0     61.999734    63.0
1     55.511378    53.0
2     70.180030    69.0
3     77.111489    81.0
4     78.752395    74.0
...         ...     ...
6004  63.683278    64.0
6005  68.934236    69.0
6006  63.340050    63.0
6007  61.613781    64.0
6008  65.614965    67.0

[6009 rows x 2 columns]


In [ ]:
# Farklı algoritmalar ile eğitme

# Decision Tree
from sklearn.tree import DecisionTreeRegressor

decision_tree = DecisionTreeRegressor(random_state=4)
decision_tree.fit(x_train, y_train)
y_pred_dtr = decision_tree.predict(x_test)

# Random Forest
from sklearn.ensemble import RandomForestRegressor

random_forest = RandomForestRegressor(random_state=15)
random_forest.fit(x_train, y_train)
y_pred_forest = random_forest.predict(x_test)

C:\Users\mucakmak\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [55]:
# Eğitilen tüm modellerin doğruluğu 
from sklearn.metrics import r2_score

print("Linear Model: ", r2_score(y_test_df, y_pred_linear))
print("Decision Tree Model: ", r2_score(y_test_df, y_pred_dtr))
print("Random Forest Model: ", r2_score(y_test_df, y_pred_forest))

Linear Model:  0.92796242597343
Decision Tree Model:  0.9275183018270137
Random Forest Model:  0.9731330470044465


In [ ]:
# En iyi doğruluk oranına sahip model ile veri setinin "Overall" sütununda boş olan yerleri doldurdum.
value = random_forest.predict(missing_data.drop("Name", axis=1))

print(missing_data["Name"])
print("Predicted Data: ", value)

18    M. ter Stegen
Name: Name, dtype: object
Predicted Data:  [89.59]
